# Baseado nos testes anteriores com esta base e com a base ChestXRay:

* Extrair features para numpy a partir da base "limpa" (usada também no notebook 01b3)
   * Rodar com maxpool e com avgpool para poder comparar
* Rodar keras_tuner e comparar resultados com melhor resultado da rede simples

### Os resultados da rede simples treinada do zero foram bons, mas caso os resultados com rede pré treinada na imagenet sejam bons, pode ser um método universal para vários classificadores, buscas e análises pré extrair features via rede pré treinada, salvando em Banco de Dados.

In [14]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import RMSprop

from utils import report_classes
import time


caminho_bases = os.path.join('..', 'bases', 'vazios_augmented2')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

MODEL_DIR = '../models_featureextraction'
LOG_DIR = '../logs/02c3/'
SIZE = (224, 224)
destino_bases = os.path.join('..', 'bases', 'vazios_featureextraction_filtered')
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
if not os.path.exists(destino_bases):
    os.mkdir(destino_bases)


# Callbacks

In [34]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02c3.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2,
                              verbose=1, mode='min')


# Model for feature extraction

In [3]:
# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='max')

# DataGenerators

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)
train_generator = train_datagen.flow_from_directory(
    caminho_train,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 41016 images belonging to 2 classes.


In [5]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

validation_generator = validation_datagen.flow_from_directory(
    caminho_test,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 4544 images belonging to 2 classes.


# Extract features to np

In [6]:
FEATURES_DIR = destino_bases

In [4]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'rb') as features_in:
    features_train = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'rb') as y_in:
    y_train = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'rb') as features_in:
    features_test = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'rb') as y_in:
    y_test = np.load(y_in)

In [7]:
def extract_features(generator, model):
    generator.reset()
    n_images = len(generator.filenames)
    batch_size = generator.batch_size
    m = batch_size *  (n_images // batch_size)  # Arredondar para não ficar espaço vazio
    n = model.output.shape[1]
    features = np.zeros((m, n), np.float32)
    y_ = np.zeros((m, 1), np.float32)
    print(n_images, m, batch_size, m // batch_size)
    for ind in range(m // batch_size):
        batch, y_batch = next(generator)
        features_batch = base_model.predict(batch)
        features[ind * batch_size: (ind * batch_size) + batch_size, :] = features_batch
        y_[ind * batch_size: (ind * batch_size) + batch_size, 0] = y_batch
    print('last batch setted elements %s:%s' % ((ind * batch_size), ((ind * batch_size) + batch_size)))
    return features, y_

In [8]:
features_train, y_train = extract_features(train_generator, base_model)

41016 40960 64 640
last batch setted elements 40896:40960


In [9]:
print(features_train.shape)
print(y_train.shape)

(40960, 1024)
(40960, 1)


In [10]:
features_test, y_test = extract_features(validation_generator, base_model)

4544 4544 64 71
last batch setted elements 4480:4544


In [11]:
print(features_test.shape)
print(y_test.shape)

(4544, 1024)
(4544, 1)


In [12]:
os.path.join(FEATURES_DIR, 'features_train.np')

'../bases/vazios_featureextraction_filtered/features_train.np'

In [15]:
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'bw') as features_out:
    np.save(features_out, features_train)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'bw') as y_out:
    np.save(y_out, y_train)

In [16]:
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'bw') as features_out:
    np.save(features_out, features_test)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'bw') as y_out:
    np.save(y_out, y_test)

# Model for feature extraction (AvgPooling)

In [17]:
# create the base pre-trained model
base_model_avg = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='avg')

# Extract features to np (AvgPooling)

In [10]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'rb') as features_in:
    features_train_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'rb') as y_in:
    y_train_avg = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'rb') as features_in:
    features_test_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'rb') as y_in:
    y_test_avg = np.load(y_in)

In [18]:
features_train_avg, y_train_avg = extract_features(train_generator, base_model_avg)

41016 40960 64 640
last batch setted elements 40896:40960


In [19]:
print(features_train_avg.shape)
print(y_train_avg.shape)

(40960, 1024)
(40960, 1)


In [20]:
features_test_avg, y_test_avg = extract_features(validation_generator, base_model_avg)

4544 4544 64 71
last batch setted elements 4480:4544


In [21]:
print(features_test_avg.shape)
print(y_test_avg.shape)

(4544, 1024)
(4544, 1)


In [22]:
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'bw') as features_out:
    np.save(features_out, features_train_avg)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'bw') as y_out:
    np.save(y_out, y_train_avg)

In [23]:
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'bw') as features_out:
    np.save(features_out, features_test_avg)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'bw') as y_out:
    np.save(y_out, y_test_avg)

# Liberar memória

In [24]:
del base_model
del base_model_avg

# Model RandomSearch KerasTuner

In [26]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu',
                           input_shape=(1024,)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Choice('rate',
                                      values=[0.2, 0.4])))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[0.0001, 0.00004, 0.00001])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [27]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='kerastunerlogs',
    project_name='Vazios')

In [28]:
tuner.search_space_summary()

# Tuner - Teste com MaxPool

In [29]:
tuner.search(x=features_train,
             y=y_train,
             epochs=10,
             validation_data=(features_test, y_test),
            )

W0711 10:36:41.924328 139874563942144 deprecation.py:323] From /home/ivan/pybr/projeto/venv/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W0711 10:37:37.166069 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.153246). Check your callbacks.
W0711 10:37:37.295309 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.150812). Check your callbacks.
W0711 10:37:37.399058 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.150812). Check your callbacks.
W0711 10:37:37.653579 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.195113). Check your callbacks.
W0711 10:37:37.679944 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.195113). Check your callbacks.
W0711 10:37:38.036134 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.225051). Check your callbacks.
W0711 10:37:38.264166 139874563942144 callbacks.py:241] Method (on_tra

W0711 10:38:49.651655 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.156142). Check your callbacks.


Name,Best model,Current model
accuracy,0.9725,0.9725
val_accuracy,0.9769,0.9769
val_loss,0.0856,0.0856
loss,0.096,0.096


W0711 10:43:28.109876 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119309). Check your callbacks.
W0711 10:43:28.129471 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119309). Check your callbacks.
W0711 10:43:28.146935 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119309). Check your callbacks.
W0711 10:43:28.164630 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119309). Check your callbacks.


...

W0711 10:44:30.771238 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.137267). Check your callbacks.


W0711 10:46:42.989434 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.222543). Check your callbacks.
W0711 10:46:43.028332 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.126002). Check your callbacks.


W0711 10:48:46.912892 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.137347). Check your callbacks.
W0711 10:48:47.057203 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.157456). Check your callbacks.
W0711 10:48:47.181962 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.157456). Check your callbacks.
W0711 10:48:47.322173 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.157975). Check your callbacks.
W0711 10:48:47.491384 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.171870). Check your callbacks.
W0711 10:48:47.619903 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.146793). Check your callbacks.
W0711 10:48:47.757087 139874563942144 callbacks.py:241] Method (on_tra

Name,Best model,Current model
accuracy,0.9725,0.9634
val_accuracy,0.9769,0.9735
val_loss,0.085,0.0982
loss,0.096,0.1293


Name,Best model,Current model
accuracy,0.9725,0.962
val_accuracy,0.9769,0.9722
val_loss,0.085,0.1023
loss,0.096,0.1375


In [30]:
tuner.results_summary()

# Tuner - Teste com AvgPool

In [31]:
tuner.search(x=features_train_avg,
             y=y_train_avg,
             epochs=10,
             validation_data=(features_test_avg, y_test_avg),
            )

W0711 10:57:56.335516 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.117355). Check your callbacks.
W0711 10:57:56.358429 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.117355). Check your callbacks.
W0711 10:57:56.461843 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.117355). Check your callbacks.
W0711 10:57:56.586942 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.121989). Check your callbacks.
W0711 10:57:56.638264 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.121989). Check your callbacks.
W0711 10:57:56.661169 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.121989). Check your callbacks.
W0711 10:57:56.780300 139874563942144 callbacks.py:241] Method (on_tra

W0711 10:58:03.945088 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.121260). Check your callbacks.
W0711 10:58:04.076936 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107612). Check your callbacks.
W0711 10:58:04.195679 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.110261). Check your callbacks.
W0711 10:58:04.218309 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107612). Check your callbacks.
W0711 10:58:04.369776 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107612). Check your callbacks.


W0711 11:01:53.845341 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.103318). Check your callbacks.
W0711 11:01:53.867009 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.103318). Check your callbacks.
W0711 11:01:53.892662 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.105550). Check your callbacks.
W0711 11:01:53.923131 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107287). Check your callbacks.
W0711 11:01:53.943017 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107287). Check your callbacks.
W0711 11:01:53.965296 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107287). Check your callbacks.


W0711 11:03:12.925754 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.104201). Check your callbacks.
W0711 11:03:13.281525 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.123613). Check your callbacks.
W0711 11:03:13.498928 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.149945). Check your callbacks.
W0711 11:03:13.623450 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.149945). Check your callbacks.
W0711 11:03:13.827644 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.179289). Check your callbacks.
W0711 11:03:14.034577 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.190520). Check your callbacks.
W0711 11:03:14.133625 139874563942144 callbacks.py:241] Method (on_tra

W0711 11:03:53.027588 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114199). Check your callbacks.
W0711 11:03:53.051122 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114199). Check your callbacks.
W0711 11:03:53.074674 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114199). Check your callbacks.
W0711 11:03:53.096460 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114199). Check your callbacks.
W0711 11:03:53.118197 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114199). Check your callbacks.


W0711 11:05:26.680376 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.130610). Check your callbacks.
W0711 11:05:26.710361 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.130610). Check your callbacks.
W0711 11:05:26.734126 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.130610). Check your callbacks.
W0711 11:05:26.755621 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.130610). Check your callbacks.


W0711 11:06:02.182270 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135204). Check your callbacks.
W0711 11:06:02.373662 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.171264). Check your callbacks.
W0711 11:06:02.560433 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.183319). Check your callbacks.
W0711 11:06:02.671280 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.183319). Check your callbacks.
W0711 11:06:03.042049 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.200401). Check your callbacks.
W0711 11:06:03.460793 139874563942144 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.200401). Check your callbacks.
W0711 11:06:03.812901 139874563942144 callbacks.py:241] Method (on_tra

Name,Best model,Current model
accuracy,0.9725,0.9698
val_accuracy,0.9769,0.9761
val_loss,0.085,0.0851
loss,0.096,0.1079


Oracle triggered exit


In [32]:
tuner.results_summary()

# Final Training and validation

In [33]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])


In [16]:
epoch = 43

In [61]:
val_loss = 0.13
model.load_weights(
    os.path.join(MODEL_DIR,
                 'Transfermodelweights02c2.{:02d}-{:.2f}.hdf5'.format(epoch, val_loss)
                ))

In [9]:
history = model.fit(x=features_train,
                    y=y_train,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test, y_test),
                    callbacks=[early_stop, mcp_save, reduce_lr, tensorboard_logs],
                   )

W0711 09:23:02.496668 140111535253248 deprecation.py:323] From /home/ivan/pybr/projeto/venv/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20800 samples, validate on 2304 samples
Epoch 1/200


W0711 09:23:04.328726 140111535253248 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.108159). Check your callbacks.


20800/20800 - 2s - loss: 0.4658 - accuracy: 0.7889 - val_loss: 0.2374 - val_accuracy: 0.9214
Epoch 2/200
20800/20800 - 0s - loss: 0.2719 - accuracy: 0.9017 - val_loss: 0.2006 - val_accuracy: 0.9319
Epoch 3/200
20800/20800 - 0s - loss: 0.2317 - accuracy: 0.9227 - val_loss: 0.1855 - val_accuracy: 0.9397
Epoch 4/200
20800/20800 - 0s - loss: 0.2097 - accuracy: 0.9319 - val_loss: 0.1757 - val_accuracy: 0.9440
Epoch 5/200
20800/20800 - 0s - loss: 0.1992 - accuracy: 0.9362 - val_loss: 0.1679 - val_accuracy: 0.9462
Epoch 6/200
20800/20800 - 0s - loss: 0.1889 - accuracy: 0.9401 - val_loss: 0.1643 - val_accuracy: 0.9462
Epoch 7/200
20800/20800 - 0s - loss: 0.1834 - accuracy: 0.9407 - val_loss: 0.1587 - val_accuracy: 0.9470
Epoch 8/200
20800/20800 - 0s - loss: 0.1783 - accuracy: 0.9437 - val_loss: 0.1557 - val_accuracy: 0.9510
Epoch 9/200
20800/20800 - 0s - loss: 0.1712 - accuracy: 0.9473 - val_loss: 0.1535 - val_accuracy: 0.9510
Epoch 10/200
20800/20800 - 0s - loss: 0.1684 - accuracy: 0.9477 - v

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [14]:
mcp_save_avg = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02c2avg.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x=features_train_avg,
                    y=y_train_avg,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test_avg, y_test_avg),
                    callbacks=[early_stop, mcp_save_avg, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.1282 - accuracy: 0.9594 - val_loss: 0.1297 - val_accuracy: 0.9583
Epoch 2/200
20800/20800 - 0s - loss: 0.1266 - accuracy: 0.9588 - val_loss: 0.1291 - val_accuracy: 0.9583
Epoch 3/200
20800/20800 - 0s - loss: 0.1250 - accuracy: 0.9588 - val_loss: 0.1293 - val_accuracy: 0.9592
Epoch 4/200

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
20800/20800 - 0s - loss: 0.1258 - accuracy: 0.9591 - val_loss: 0.1295 - val_accuracy: 0.9579
Epoch 5/200
20800/20800 - 0s - loss: 0.1231 - accuracy: 0.9601 - val_loss: 0.1279 - val_accuracy: 0.9588
Epoch 6/200
20800/20800 - 0s - loss: 0.1213 - accuracy: 0.9600 - val_loss: 0.1273 - val_accuracy: 0.9588
Epoch 7/200
20800/20800 - 0s - loss: 0.1217 - accuracy: 0.9596 - val_loss: 0.1276 - val_accuracy: 0.9583
Epoch 8/200

Epoch 00008: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
20800/20800 - 0s - loss: 0.1212 - accurac

In [44]:
y_pred = model.predict(features_test)

y_pred = y_pred > 0.5

classes = {0: 'nvazio', 1: 'vazio'}
report_classes(classes, y_test, y_pred)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 1077 	 89
vazio 	 11 	 1127
              precision    recall  f1-score   support

         0.0       0.99      0.92      0.96      1166
         1.0       0.93      0.99      0.96      1138

    accuracy                           0.96      2304
   macro avg       0.96      0.96      0.96      2304
weighted avg       0.96      0.96      0.96      2304



In [49]:
y_pred_train = model.predict(features_train)

y_pred_train = y_pred_train > 0.5

report_classes(classes, y_train, y_pred_train)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 9745 	 749
vazio 	 40 	 10266
              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96     10494
         1.0       0.93      1.00      0.96     10306

    accuracy                           0.96     20800
   macro avg       0.96      0.96      0.96     20800
weighted avg       0.96      0.96      0.96     20800



In [62]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
mcp_save_weights = ModelCheckpoint(
    os.path.join(MODEL_DIR, 'Transfermodelweights02c2-classweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                 save_best_only=True, monitor='val_loss', mode='min'
)
class_weight = {0: 3., 1: 1.}
history = model.fit(x=features_train,
                    y=y_train,
                    batch_size=128,
                    class_weight=class_weight,
                    initial_epoch=0,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test, y_test),
                    callbacks=[early_stop, mcp_save_weights, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.2577 - accuracy: 0.9542 - val_loss: 0.1488 - val_accuracy: 0.9540
Epoch 2/200
20800/20800 - 0s - loss: 0.2534 - accuracy: 0.9544 - val_loss: 0.1513 - val_accuracy: 0.9544
Epoch 3/200

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
20800/20800 - 0s - loss: 0.2475 - accuracy: 0.9556 - val_loss: 0.1516 - val_accuracy: 0.9540
Epoch 4/200
20800/20800 - 0s - loss: 0.2496 - accuracy: 0.9544 - val_loss: 0.1532 - val_accuracy: 0.9536
Epoch 5/200
20800/20800 - 1s - loss: 0.2474 - accuracy: 0.9543 - val_loss: 0.1484 - val_accuracy: 0.9531
Epoch 6/200
20800/20800 - 1s - loss: 0.2490 - accuracy: 0.9550 - val_loss: 0.1498 - val_accuracy: 0.9540
Epoch 7/200

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
20800/20800 - 1s - loss: 0.2470 - accuracy: 0.9551 - val_loss: 0.1521 - val_accuracy: 0.9527
Epoch 8/200
20800/20800 - 1s - loss: 0.2475 - accuracy

In [63]:
y_pred = model.predict(features_test)

y_pred = y_pred > 0.5

classes = {0: 'nvazio', 1: 'vazio'}
report_classes(classes, y_test, y_pred)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 1095 	 71
vazio 	 36 	 1102
              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95      1166
         1.0       0.94      0.97      0.95      1138

    accuracy                           0.95      2304
   macro avg       0.95      0.95      0.95      2304
weighted avg       0.95      0.95      0.95      2304



In [64]:
y_pred_train = model.predict(features_train)

y_pred_train = y_pred_train > 0.5

report_classes(classes, y_train, y_pred_train)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 9918 	 576
vazio 	 217 	 10089
              precision    recall  f1-score   support

         0.0       0.98      0.95      0.96     10494
         1.0       0.95      0.98      0.96     10306

    accuracy                           0.96     20800
   macro avg       0.96      0.96      0.96     20800
weighted avg       0.96      0.96      0.96     20800

